## N-puzzle : problem statement

The n-puzzle is a classic problem often used to model algorithms that involve heuristics. Two commonly used heuristics for this problem are:  

- **Misplaced Tiles**: Counting the number of tiles that are not in their correct position in the goal configuration.  
- **Manhattan Distance**: Calculating the sum of the taxicab distances (vertical and horizontal moves) required for each tile to reach its target position in the goal configuration.  

Both heuristics are admissible, meaning they never overestimate the actual cost to solve the puzzle.


In [1]:
import numpy as np
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
from icecream import ic
from dataclasses import dataclass

/Users/maot/projects/computational_intelligence/lab3/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
PUZZLE_DIM = 3
action = namedtuple('Action', ['pos1', 'pos2'])

In [16]:
def action_available(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def goal_test(state: np.ndarray) -> bool:
    c = 1
    for i in state:
        for j in i:
            if j != c:
                return False
            c += 1
            if c == len(state)**2:
                c = 0
    return True

def state_listed(state: np.ndarray, frontier: list[np.ndarray]) -> int:
    for i, el in enumerate(frontier):
        c = 0
        for x in range(PUZZLE_DIM):
            for y in range(PUZZLE_DIM):
                if state[x][y] == el[x][y]:
                    c += 1
        if c == PUZZLE_DIM**2:
            return i
    return -1


In [17]:
RANDOMIZE_STEPS = 100_000
initial_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    #scelgo randomicamente un azione da quelle possibili e la effettuo
    initial_state = do_action(initial_state, choice(action_available(initial_state)))
initial_state

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 317018.76it/s]


array([[3, 4, 7],
       [8, 1, 2],
       [0, 5, 6]])

## Depth-First Search approach


Depth-First Search (DFS) is a fundamental uninformed search algorithm that explores as far as possible along each branch before backtracking. It is particularly suited for problems where the solution lies deep in the search tree.

- **Exploration Strategy**: DFS dives deep into a branch until it reaches a goal state or a dead end, at which point it backtracks to explore other branches.  
- **Data Structure**: DFS typically uses a stack (explicit or via recursion) to keep track of the nodes to visit.  

## Steps in the DFS Algorithm

1. **Initialization**: Start with the initial state and add it to a stack.  
2. **Expand Nodes**: Pop a node from the stack and mark it as visited.  
3. **Goal Check**: If the current node is the goal state, terminate the search.  
4. **Add Neighbors**: Push all unvisited neighbors of the current node onto the stack.  
5. **Repeat**: Continue until the goal is found or the stack is empty (indicating no solution exists).  

## Key Features

- **Memory Usage**: DFS uses less memory compared to breadth-first search (BFS) because it only stores the current branch in the stack.  
- **Non-Optimal Solutions**: DFS does not guarantee the shortest or optimal path to the goal, especially in unweighted graphs.  
- **Completeness**: In finite graphs, DFS will find a solution if one exists, but it may fail in infinite graphs if it gets stuck exploring an infinitely deep branch.  
- **Variants**: Iterative Deepening Depth-First Search (IDDFS) combines the benefits of DFS and BFS to ensure completeness while limiting memory usage.


In [22]:
cost = 0
frontier = list()
visited_state = list()
cost_list = list()
state = initial_state
for step in tqdm(range(10000)):
    visited_state.append(state)
    if goal_test(state):
        break
    for a in action_available(state):
        new_state = do_action(state, a)
        new_cost = cost + 1
        if state_listed(new_state, visited_state) == -1 and state_listed(new_state, frontier) == -1:
            cost_list.append(new_cost)
            frontier.append(new_state)
    state = frontier.pop()
    cost = cost_list.pop()
efficiency = cost / (len(frontier) + len(visited_state))
ic(goal_test(state), state, efficiency)
None

100%|██████████| 10000/10000 [08:19<00:00, 20.01it/s]
ic| goal_test(state): False
    state: array([[8, 3, 0],
                  [5, 1, 7],
                  [2, 6, 4]])
    efficiency: 0.5499575911789653


## Results
High memory consumption poses a significant challenge, and solutions cannot always be found if the number of iterations is insufficient.  

- **8-puzzle**: Requires excessive time (> 15 minutes)  
- **15-puzzle**: Computationally infeasible due to excessive time requirements  


## A*

A* is a widely-used informed search algorithm that combines two factors to prioritize the most promising states for exploration:

- **g-score**: The actual cost of reaching the current state from the start state.  
- **h-score**: A heuristic estimate of the remaining cost to reach the goal state. In this case, the heuristic is the **Manhattan Distance**, which sums the vertical and horizontal distances between each tile's current position and its target position.  
- **f-score**: The total estimated cost, calculated as \( f = g + h \). States with the lowest f-score are prioritized for expansion.  

### Steps in the A* Algorithm

1. **Initialization**: Begin with the initial scrambled state.  
2. **Priority Queue**: Use a priority queue (min-heap) to manage states, with the f-score as the priority.  
3. **Expand States**: Remove the state with the lowest f-score from the queue, compute its neighbors, and add them to the queue.  
4. **Goal Check**: Repeat the process until the goal state is reached or no solution exists.  

### Key Features

- **Optimal Solution**: A* guarantees an optimal solution with the fewest moves, provided the heuristic is admissible (never overestimates the true cost).  
- **Memory Usage**: A* is memory-intensive as it stores all explored states and their associated f-scores.


In [ ]:
@dataclass
class Node:
    state: np.ndarray
    cost: int = 0
    quality: int = 0

def stateInNodeList(state: np.ndarray, frontier: list[Node]) -> int:
    for i, el in enumerate(frontier):
        c = 0
        for x in range(PUZZLE_DIM):
            for y in range(PUZZLE_DIM):
                if state[x][y] == el.state[x][y]:
                    c += 1
        if c == PUZZLE_DIM**2:
            return i
    return -1


def manhattan_dist(state: np.ndarray) -> int:
    counter = 0
    c = 1
    for x in range(0, PUZZLE_DIM):
        for y in range(0, PUZZLE_DIM):
            if state[x][y] != c and state[x][y] != 0:
                counter += abs(x - ((state[x][y] - 1) // PUZZLE_DIM)) + abs(y - ((state[x][y] - 1) % PUZZLE_DIM))
            c += 1
    return counter

def linear_conflict(state: np.ndarray) -> int:
        conflict = 0
        size = len(state)
        # Row conflicts
        for row in range(size):
            max_val = -1
            for col in range(size):
                value = state[row][col]
                if value != 0 and (value - 1) // size == row:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2
        # Column conflicts
        for col in range(size):
            max_val = -1
            for row in range(size):
                value = state[row][col]
                if value != 0 and (value - 1) % size == col:
                    if value > max_val:
                        max_val = value
                    else:
                        conflict += 2
        return conflict

In [ ]:
frontier = list()
visited_state = list()
state = Node(initial_state, 1 + manhattan_dist(initial_state) + linear_conflict(initial_state), 0)
for step in tqdm(range(10000)):
    visited_state.append(state)
    if state.cost == state.quality:
        break
    for a in action_available(state.state):
        new_state = do_action(state.state, a)
        if stateInNodeList(new_state, visited_state) == -1 and stateInNodeList(new_state, frontier) == -1:
            frontier.append(Node(new_state, state.quality + 1 + manhattan_dist(new_state) + linear_conflict(new_state), state.quality + 1))
    frontier.sort(key=lambda i: i.cost, reverse = False)
    state = frontier.pop(0)
efficiency = state.cost / (len(frontier) + len(visited_state))
ic(initial_state, manhattan_dist(initial_state) + linear_conflict(initial_state), goal_test(state.state), state, efficiency)
None

  4%|▍         | 385/10000 [00:00<00:15, 625.96it/s] 
ic| initial_state: array([[3, 4, 7],
                          [8, 1, 2],
                          [0, 5, 6]])
    ManhattanDistance(initial_state) + LinearConflict(initial_state): np.int64(16)
    goal_test(state.state): True
    state: Node(state=array([[1, 2, 3],
                  [4, 5, 6],
                  [7, 8, 0]]),
                cost=22,
                quality=22)
    efficiency: 0.03542673107890499


## Results
For puzzles with a low \( N \), solutions can typically be found within a reasonable number of iterations. However, as the puzzle's size increases, finding a solution becomes significantly more challenging, and may not always be feasible.  

### Performance Comparison of Cost Functions

- **Hamming Distance**:  
  - **8-puzzle**: Time = 7m 9s, Quality = 22  
  - **15-puzzle**: Computationally infeasible due to excessive time requirements  

- **Manhattan Distance**:  
  - **8-puzzle**: Time = 2.8s, Quality = 22  
  - **15-puzzle**: Computationally infeasible due to excessive time requirements  

- **Manhattan Distance + Linear Conflict**:  
  - **8-puzzle**: Time = 1.4s, Quality = 22  
  - **15-puzzle**: Computationally infeasible due to excessive time requirements  
